<a href="https://colab.research.google.com/github/petteriTeikari/minivess_mlops/blob/main/notebooks/train_monai_model_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use a custom model quickly

Reuse the data(loader) from the "ML pipe"

Check if this Notebook is run on Colab or not [Test if notebook is running on Google Colab](https://stackoverflow.com/a/63519730/6412152)

In [6]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
print(f'Running in Colab = {RunningInCOLAB}')

Running in Colab = True


Colab-specific installs [Python Poetry Project on Colab](https://gist.github.com/DPaletti/da7729c35ba7f9274c3635849608b7bc?permalink_comment_id=4455088)

In [2]:
import os
os.chdir("/content")

!git clone https://github.com/petteriTeikari/minivess_mlops.git


Cloning into 'minivess_mlops'...
remote: Enumerating objects: 1432, done.
remote: Counting objects: 100% (373/373), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 1432 (delta 194), reused 258 (delta 122), pack-reused 1059
Receiving objects: 100% (1432/1432), 754.70 KiB | 5.51 MiB/s, done.
Resolving deltas: 100% (849/849), done.


In [3]:
!pip install poetry
os.chdir("/content/minivess_mlops")
!poetry config virtualenvs.in-project false
!poetry install

Streaming output truncated to the last 5000 lines.
  • Installing nvidia-cuda-nvrtc-cu11 (11.7.99): Installing...
  • Installing nvidia-cuda-runtime-cu11 (11.7.99)
  • Installing nvidia-cudnn-cu11 (8.5.0.96): Downloading... 2%
  • Installing nvidia-cufft-cu11 (10.9.0.58): Downloading... 6%
  • Installing nvidia-curand-cu11 (10.2.10.91): Downloading... 20%
  • Installing itk-filtering (5.3.0): Downloading... 79%
  • Installing itsdangerous (2.1.2)
  • Installing joblib (1.3.2)
  • Installing jupyterlab (4.0.7)
  • Installing jupyterlab-widgets (3.0.9)
  • Installing mako (1.2.4)
  • Installing matplotlib (3.7.3)
  • Installing nvidia-cublas-cu11 (11.10.3.66): Downloading... 14%
  • Installing nvidia-cuda-cupti-cu11 (11.7.101)
  • Installing nvidia-cuda-nvrtc-cu11 (11.7.99): Installing...
  • Installing nvidia-cuda-runtime-cu11 (11.7.99)
  • Installing nvidia-cudnn-cu11 (8.5.0.96): Downloading... 2%
  • Installing nvidia-cufft-cu11 (10.9.0.58): Downloading... 6%
  • Installing nvidia-cur

Import the libraries needed

In [5]:
import sys
from tqdm import tqdm
from loguru import logger

from src.run_training import parse_args_to_dict
from src.training.experiment import define_experiment_data
from src.utils.config_utils import import_config

ModuleNotFoundError: ignored

Import helper subfunction(s)

In [ ]:
def get_dataloaders(experim_dataloaders: dict):
    # Get the "validation" and "train" dataloaders from the dictionary
    fold_name = 'fold0'
    split_names = list(experim_dataloaders[fold_name].keys())
    fold_key = experim_dataloaders.get(fold_name)
    if fold_key is not None:
        try:
            train = experim_dataloaders[fold_name]['TRAIN']
            val = experim_dataloaders[fold_name]['VAL']['MINIVESS']
        except Exception as e:
            raise IOError('Could not get the dataloaders from the dictionary, error = {}'.format(e))
    else:
        raise IOError('Fold name = "{}" not found in the dataloaders dictionary'.format(fold_name))
    return train, val

Input arguments for the training (you can add all the input arguments supported by `run_training.py` here

In [ ]:
input_args = ['-c', 'tutorials/train_demo']

# Fake these as coming from the command line to match the main code (run_training.py)
sys.argv = ['notebook_run']  # Jupyter has all the extra crap, so replace that with this
for sysargv in input_args:
    sys.argv.append(sysargv)
args = parse_args_to_dict()

Create the config with Hydra from the .yaml file(s)

In [ ]:
config, exp_run = import_config(args=args, task_cfg_name=args['task_config_file'])

Import the dataloaders (now the data augmentations are here as well as data transformations)

In [ ]:
_, _, experim_dataloaders, exp_run = (
        define_experiment_data(config=config,
                               exp_run=exp_run))

# Get the "validation" and "train" dataloaders from the dictionary
train, val = get_dataloaders(experim_dataloaders)

Now you are ready to train your new model that you just wanna quickly test without
wanting to have a battle with the config .YAML files
Add maybe some fastai demo with MLflow autologging:
https://github.com/mlflow/mlflow/blob/master/examples/fastai/train.py

In [ ]:
# Iterate the dataloaders for demo
no_of_epochs = 3
logger.info('Training for {} epochs'.format(no_of_epochs))
for epoch in range(no_of_epochs):

    logger.info('Epoch {}/{}'.format(epoch, no_of_epochs - 1))

    # Train
    logger.info('train with {} batches'.format(len(train)))
    for i, batch in enumerate(train):
        images, mask = batch['image'], batch['label']

    # Validation
    logger.info('validate with {} batches'.format(len(train)))
    for j, batch in enumerate(val):
        images, mask = batch['image'], batch['label']

logger.info('Training done!')